In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np

### X

In [ ]:
df = pd.read_csv("../../data/perturb/clustered_mean_gene_expression_figs2-4.csv", sep=",", index_col=0, low_memory=False)
df = df.iloc[2:, 1:]

# convert perturb experiment name to perturb gene name
df.columns = pd.DataFrame(df.columns)[0].str.split("_", expand=True)[1].values

# take mean of two experiment if perturb genes is the same
df = df.T
df.index.name = 'perturb'
df = df.groupby('perturb').mean()
df = df.sort_index()
col_names = list(df.columns)
col_names.sort()
df = df.loc[:, col_names]

# the perturbed gene is set to have 0 expression
df = df.fillna(0).T

In [ ]:
# sort both axis by gene name
df.index.name = None
df.columns.name = None
df = df.sort_index()
df = df.T
df = df.sort_index()
df = df.T

In [ ]:
df.to_csv("../../result/input_perturb_expression/X", sep="\t")

### Y

In [ ]:
gene_names = df.index.values

data = sc.read_h5ad("../../data/perturb/K562_gwps_normalized_bulk_01.h5ad")
ctrl_id = list(data.obs[data.obs['core_control']].index)

# take mean of ctrl experiments
ctrl_exp = data.to_df().loc[ctrl_id, :].mean()
ctrl_exp.index = ctrl_exp.index.map(data.var['gene_name'].to_dict())
Y = ctrl_exp[gene_names].groupby('gene_id').mean()[gene_names].to_frame()
Y.index.name = "gene"
Y.columns = ["wt_exp"]

In [ ]:
Y.to_csv("../../result/input_perturb_expression/Y", sep="\t")